In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('spam.csv', encoding='latin1')

In [3]:
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,ham,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


In [4]:
y = data['v1'].values
X = data['v2'].values


In [5]:
from sklearn.model_selection import train_test_split
import regex as re
import spacy
import en_core_web_sm

In [6]:
def remove_punctuation(text):
    return re.sub(r'\p{P}+', "", text)
def change_num(text):
    return re.sub(r'[\d]+', "1", text)    

In [7]:
nlp = en_core_web_sm.load()
def lemmatization(text):
    tx = change_num(remove_punctuation(text.lower()))
    doc = nlp(tx)
    return [token.lemma_ for token in doc]

In [8]:
X_lem = [lemmatization(text) for text in X]

In [21]:
type(y_train)

numpy.ndarray

In [98]:
#y_train = np.array(y_train)
#X_train = np.array(X_train)
#X_train[y_train == 'ham']

In [99]:
from collections import defaultdict

In [100]:
def train(X,y):
    count_word_in_class = defaultdict(int)#
    count_class = defaultdict(int)#
    uniq_word = defaultdict(int)
    word_count = defaultdict(int)
    for item in zip(X,y):
        count_class[item[1]] +=1
        for word in item[0]:
            word_count[item[1],word] += 1
            count_word_in_class[item[1]] += 1
            uniq_word[word] += 1
    return count_word_in_class, count_class, uniq_word,word_count

In [157]:
def test(X,y, count_word_in_class, count_class, uniq_word, word_count, z=1,k=1):
    ham = np.log(count_class['ham']/(k*count_class['spam']+count_class['ham']))
    spam = np.log(k*count_class['spam']/(k*count_class['spam']+count_class['ham']))
    y_pred =[]
    for item in zip(X,y):
        P_ham = ham
        P_spam = spam
        for word in item[0]:
            P_ham += np.log( (word_count[('ham', word)]+z)/(count_word_in_class['ham']+ z*len(uniq_word)) )
            P_spam += np.log( (k*word_count[('spam', word)]+z)/(k*count_word_in_class['spam']+ z*len(uniq_word)) )
        y_pred.append([P_ham,P_spam])
    r =0
    pr =[]
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for y1, y2 in zip(y_pred,y):
        y_ham = y1[0]
        y_spam = y1[1]
        #,slkjrjl
        if y_ham < y_spam and y2 == 'spam':
            TP += 1
        if y_ham < y_spam and y2 != 'spam':
            FP += 1
            pr.append(((y_ham,y_spam),y2))
        if y_ham > y_spam and y2 == 'ham':
            TN += 1
        if y_ham > y_spam and y2 != 'ham':
            FN += 1
            pr.append(((y_ham,y_spam),y2))
        
    precision = TP/(TP + FP)
    reccal = TP/(TP + FN)
    F1 = 2*precision*reccal/(precision + reccal)
    return precision,reccal,F1

In [166]:
X_train,X_test,y_train,y_test = train_test_split(X_lem, y, stratify=y,  test_size = 0.25, random_state = 1)

In [167]:
count_word_in_class, count_class, uniq_word, word_count = train(X_train,y_train)

In [180]:
X_train

defaultdict(int, {'ham': 3619, 'spam': 560})

In [168]:
k = count_class['ham']/count_class['spam']

In [169]:
k = int(k)
k

6

In [176]:
precision,reccal,F1 = test(X_test,y_test,count_word_in_class, count_class, uniq_word, word_count,2,k)
print(precision,reccal,F1)

0.8660287081339713 0.9679144385026738 0.9141414141414141


In [175]:
print(precision,reccal,F1)

0.8979591836734694 0.9411764705882353 0.9190600522193212
